In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("GOOG.csv")

In [3]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,52.239193,15319700
4,2004-08-25,52.284027,53.798351,51.746044,52.802086,52.802086,9232100
...,...,...,...,...,...,...,...
3804,2019-09-30,1220.969971,1226.000000,1212.300049,1219.000000,1219.000000,1404100
3805,2019-10-01,1219.000000,1231.229980,1203.579956,1205.099976,1205.099976,1273500
3806,2019-10-02,1196.979980,1196.979980,1171.290039,1176.630005,1176.630005,1615100
3807,2019-10-03,1180.000000,1189.060059,1162.430054,1187.829956,1187.829956,1621200


In [4]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(0,1))
Data=scaler.fit_transform(df[["Open","High","Low","Close","Adj Close","Volume"]])

In [5]:
n_step=30


x,y=[],[]
for i in range(n_step,len(Data)):
    x.append(Data[i-n_step:i])
    y.append(Data[i])

In [8]:
import numpy as np
x=np.array(x)
y=np.array(y)

In [9]:
from sklearn.model_selection import train_test_split

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)

In [11]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [17]:
model = Sequential([
    LSTM(50,input_shape=(xtrain.shape[1],xtrain.shape[2])),
    Dropout(0.2),
    Dense(units=ytrain.shape[1],activation="linear")
])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                11400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 306       
Total params: 11,706
Trainable params: 11,706
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.compile(optimizer="adam",loss="mean_squared_error")

In [20]:
model.fit(xtrain,ytrain,epochs=10,batch_size=32,validation_data=(xtest,ytest))

Train on 3023 samples, validate on 756 samples
Epoch 1/10
3023/3023 [==============================] - 11s 4ms/sample - loss: 0.0174 - val_loss: 0.0016
Epoch 2/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0046 - val_loss: 0.0011
Epoch 3/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0036 - val_loss: 7.1040e-04
Epoch 4/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0031 - val_loss: 6.5003e-04
Epoch 5/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0029 - val_loss: 6.6973e-04
Epoch 6/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0024 - val_loss: 8.8539e-04
Epoch 7/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0024 - val_loss: 7.2056e-04
Epoch 8/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0022 - val_loss: 7.3703e-04
Epoch 9/10
3023/3023 [==============================] - 5s 2ms/sample - loss: 0.0020 - val_loss:

In [21]:
TestL=model.evaluate(xtest,ytest)
TestL

756/756 [==============================] - 0s 376us/sample - loss: 5.9296e-04


0.000592964049367106

In [23]:
pred=model.predict(xtest)

pred=scaler.inverse_transform(pred)

In [24]:
ytestInv=scaler.inverse_transform(ytest)

In [25]:
Date=df.Date.values[n_step+len(xtrain):]

In [26]:
res= pd.DataFrame({
    "date":Date,
    "Pred":pred[:,3],
    "actual":ytestInv[:,3]
})

res

,date,Pred,actual
0,2016-10-04,277.437469,284.214691
1,2016-10-05,538.405701,537.474365
2,2016-10-06,570.642517,584.274963
3,2016-10-07,203.498672,201.514679
4,2016-10-10,163.953949,164.353958
...,...,...,...
751,2019-09-30,96.861610,92.254211
752,2019-10-01,759.994629,747.919983
753,2019-10-02,1196.338623,1193.000000
754,2019-10-03,705.145264,698.210022
